# Setup of Noteboook

The follwing code clones the github repository with course files. 
Subsequently it imports all libraries and custom modules needed for this notebook

In [1]:
# clone the github repository
!git clone https://github.com/DataHow/analytics-course-scripts.git

Cloning into 'analytics-course-scripts'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 336 (delta 63), reused 65 (delta 30), pack-reused 236
Receiving objects: 100% (336/336), 5.36 MiB | 15.13 MiB/s, done.
Resolving deltas: 100% (197/197), done.


In [2]:
# import libaries
import pandas as pd
import numpy as np
import scipy
import importlib  
import itertools
import scipy.integrate
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib import pyplot as plt
pd.set_option('display.max_rows', None)

# import custom modules
simulator = importlib.import_module("analytics-course-scripts.scripts.modules.simulator")
plothelpers = importlib.import_module("analytics-course-scripts.scripts.modules.plothelpers")

# Introduction to Cell Culture Fed-Batch Process Simulator

The simulator is aimed at providing in-silico data to test some of the machine learning tool discussed during the course. The simulator is mimicking the behavior of a fed-batch cell culture process, where only few components are present:


*   The cells, responsible for the production of the product, which are consuming glucose to sustain metabolism and producing lactate as by-product. These are indicated by VCD (viable cell density), typically expressed in million cells / ml.
*   Glucose (Glc) is consumed by the cells and it is continuously fed to the process (F_glc). Too low glucose concentrations are slowing down cell growth and product production. To high glucose concentrations are poisoning the system and accelerating cell death.
*   Lactate (Lac) is a by-product of the cells and it is poisoning the cells, so that too high lactate concentrations are slowing down cell metabolism and accelerating cell death
*   Product (Titer) is produced by the cells. The faster the cells are growing, the less are producing the product. 

The simulator's results are based on the following equations to create the in-sillico data

### Model Equation Parameters

- Balance on VCD:  $\frac{dVCD}{dt}$ = (μ<sub>g</sub> - μ<sub>d</sub>)VCD
- Balance on Glucose: $\frac{dGlc}{dt}$ = $-k_{Glc} \frac{Glc}{Glc + 0.05}$ VCD + F<sub>Glc</sub>
- Balance on Lactate: $\frac{dLac}{dt}$ = k<sub>Lac</sub> VCD
- Balance on Product (Titer): $\frac{dProd}{dt}$ = k<sub>Prod</sub>$\frac{Glc}{Glc + K_{g, Glc} }$ ($\frac{μ_{g}}{μ_{g,max}}$)<sup>2</sup> VCD $-2 \frac{dAggr}{dt}$ 

Where:
- Growth rate: $μ_{g} =  μ_{g,max}\frac{Glc}{Glc +K_{g, Glc}}\frac{K_{i, Lac}}{Lac+K_{i, Lac}} $
- Death rate: $μ_{d} = μ_{d,max}(1+\frac{φ}{1+φ})\frac{Lac}{Lac+K_{d, Lac}}$
- Glc saturation: φ = e<sup>0.1(Glc-75)</sup>

The user can change the different rates on the simulator, in order to change the process behavior (please use default values at the beginning)


### Process Parameters

Please insert the values of the process manipulated variables:

- Feed start (day): day at which Glc feed is started
- Feed end (day): ay at which Glc feed is stopped
- Feed rate: mass rate (g/L/day) at which Glc is feed (continuous feed over 24hours)
- Initial Glc concentration (g/L): Glc at time t = 0
- Initial VCD (10^6 cell/mL): VCD at time t = 0

# Generate Experiment Run

In [3]:
""" Model parameters: Dictate the behaviour of the cell process """
MU_G_MAX = 0.05;
MU_D_MAX = 0.025;
K_G_GLC  = 1;
K_I_LAC  = 30;
K_D_LAC  = 50;
K_GLC    = 0.04;
K_LAC    = 0.06;
K_PROD   = 1;
MODEL_PARAM = [MU_G_MAX,MU_D_MAX,K_G_GLC,K_I_LAC,K_D_LAC,K_GLC,K_LAC,K_PROD]

""" Process parameters: Conditions at which process is run """
FEED_START = 3.0
FEED_END = 12.0
GLC_FEED_RATE = 12.0
GLC_0 = 10.0
VCD_0 = 0.3
PROCESS_PARAM = [FEED_START,FEED_END,GLC_FEED_RATE,GLC_0,VCD_0]


In [4]:
# Generate experiment run
t,x = simulator.predict_process(MODEL_PARAM,PROCESS_PARAM)
times = np.array(t)
run = pd.DataFrame(x,columns=["X:VCD", "X:Glc", "X:Lac", "X:Titer"])
run.head(15)

,X:VCD,X:Glc,X:Lac,X:Titer
0,0.300000,10.000000,0.000000,0.000000
1,0.877666,9.484266,0.777573,0.129431
2,2.389344,8.030174,2.971193,0.931287
3,5.451007,4.419283,8.432332,7.038657
4,9.954081,9.171630,19.380135,32.672120
5,14.236051,9.490923,36.991679,107.606442
6,16.097601,6.789103,59.179200,249.179059
7,15.297378,3.687971,82.056450,427.837006
8,12.870307,2.335591,102.445937,595.285539
9,10.245350,3.474163,119.046093,738.830470


In [5]:
# Plot generated experiment run
fig = make_subplots(rows=1, cols=4, subplot_titles=run.columns)
for i,column in enumerate(run.columns):
    fig.add_trace(go.Scatter(x=times/24, y=run[column].values), row=1, col=i+1)
fig.update_layout(showlegend=False, title_text="X Variable evolution for generated run")
fig.show()

# Generate Design of Experiments

In order to show the complexity of the model, in spite of the apparent simplicity and small number of components, in this section the user can similate the behavior of the process in a broad range of the process variables.

A number of simulations defined by "num_runs" will be generated. For this number of simulations, a latin hypercube design (LHD) is created, to uniformly map the 5-dimensional space of the variables.

### Manipulated Variables

For each of the manipulated variables defined in the section above, the use can define the limits of the exploration space (first value: lower limit; second value: upper limit).

*Note: num_runs is defining the number of simulations. The generated simulations are saved locally in the file "ExperimentsDOE_test.csv", which will be used as test set for the machine learnign tools in the followng lectures.*




In [6]:
""" DOE Dataset definition (variable = [lower bound, upper bound]) """
""" Model parameters: Dictate the behaviour of the cell process """
MU_G_MAX = 0.05;
MU_D_MAX = 0.025;
K_G_GLC  = 1;
K_I_LAC  = 30;
K_D_LAC  = 50;
K_GLC    = 0.04;
K_LAC    = 0.06;
K_PROD   = 1;

""" Process parameters: Conditions at which process is run """
FEED_START = [1, 4]
FEED_END = [8, 12]
GLC_FEED_RATE = [5, 20]
GLC_0 = [10, 80.0]
VCD_0 = [0.1, 1.0]

""" Number of experiments to generate """
NUM_RUNS = 40

""" Filename and filepath for the dataset """
FILENAME = "owu.csv"
FILEPATH = "/content/"

# Collect parameters to dictionary
VAR_LIMS = {"mu_g_max":MU_G_MAX,
    "mu_d_max": MU_D_MAX,
    "K_g_Glc" : K_G_GLC,
    "K_I_Lac" : K_I_LAC,
    "K_d_Lac" : K_D_LAC,
    "k_Glc" : K_GLC,
    "k_Lac" : K_LAC,
    "k_Prod" : K_PROD,
    "feed_start" : FEED_START,
    "feed_end" : FEED_END,
    "Glc_feed_rate" : GLC_FEED_RATE,
    "Glc_0" : GLC_0,
    "VCD_0" : VCD_0}

In [7]:
# Generate Dataset
data = simulator.generate_data(VAR_LIMS, NUM_RUNS, FILENAME)
# Import DOE
doe = pd.read_csv(FILEPATH+FILENAME.replace(".csv","_doe.csv"),index_col=None, usecols =["feed_start","feed_end","Glc_feed_rate","Glc_0","VCD_0"])
# Import OWU
owu = pd.read_csv(FILEPATH+FILENAME,index_col=None, usecols = ["X:VCD", "X:Glc", "X:Lac", "X:Titer","W:Feed"])
owu.index = pd.MultiIndex.from_product([list(range(NUM_RUNS)),list(range(15))], names=["run","time"])

In [8]:
# Read DOE - Initial Conditions
doe

,feed_start,feed_end,Glc_feed_rate,Glc_0,VCD_0
0,2.500000,10.000000,12.500000,45.000000,0.550000
1,1.269231,11.230769,19.038462,52.179487,0.111538
2,1.576923,10.410256,5.961538,77.307692,0.388462
3,2.423077,9.076923,19.423077,39.615385,0.596154
4,1.192308,8.871795,5.576923,53.974359,0.550000
5,1.038462,10.615385,7.115385,79.102564,0.826923
6,3.807692,8.564103,12.884615,23.461538,0.157692
7,1.807692,11.435897,10.961538,28.846154,0.526923
8,2.653846,9.589744,15.192308,12.692308,0.965385
9,2.192308,9.179487,19.807692,14.487179,0.342308


In [9]:
# Read OWU - Process Evolutions
owu

X:VCD         X:Glc       X:Lac      X:Titer     W:Feed
run time                                                             
0   0      0.550000  4.500000e+01    0.000000     0.000000   0.000000
    1      1.725244  4.400819e+01    1.489389     0.054632   0.000000
    2      4.779558  4.108962e+01    5.872389     1.311539  12.500000
    3     10.278397  4.027148e+01   16.487303    15.870869  12.500000
    4     15.886799  4.008360e+01   35.542541    85.824621  12.500000
    5     18.568089  3.578307e+01   60.776450   245.221131  12.500000
    6     18.099141  3.050520e+01   87.483607   468.616199  12.500000
    7     15.811653  2.666724e+01  112.033694   706.666289  12.500000
    8     12.920362  2.538948e+01  132.740373   924.761704  12.500000
    9     10.123559  2.687354e+01  149.296217  1108.215155  12.500000
    10     7.715657  3.086152e+01  162.086471  1254.876222   0.000000
    11     5.775454  2.443517e+01  171.743604  1367.869866   0.000000
    12     4.271029  1.965583e+01  178.928939  1452.631393   0.000000
    13     3.129193  1.613956e+01  184.218131  1515.192811   0.000000
    14     2.276241  1.357394e+01  188.079549  1560.822799   0.000000
1   0      0.111538  5.217949e+01    0.000000     0.000000   0.000000
    1      0.359731  5.197594e+01    0.305615     0.002874  19.038462
    2      1.127734  6.524181e+01    1.277170     0.032104  19.038462
    3      3.227071  8.234044e+01    4.188852     0.488335  19.038462
    4      7.297872  9.646293e+01   11.566872     6.303100  19.038462
    5     11.396314  1.063827e+02   25.251686    39.874716  19.038462
    6     12.423133  1.137009e+02   42.839929   122.762747  19.038462
    7     10.553484  1.215616e+02   59.613716   233.785401  19.038462
    8      7.724744  1.318196e+02   72.789730   337.585523  19.038462
    9      5.192403  1.447080e+02   82.018141   417.307470  19.038462
    10     3.324652  1.597138e+02   88.069067   472.302288  19.038462
    11     2.069966  1.762066e+02   91.888798   508.034816   0.000000
    12     1.267869  1.790286e+02   94.247449   530.471205   0.000000
    13     0.769060  1.780704e+02   95.685132   544.280195   0.000000
    14     0.463804  1.774909e+02   96.554670   552.680124   0.000000
2   0      0.388462  7.730769e+01    0.000000     0.000000   0.000000
    1      1.239286  7.660062e+01    1.061295     0.017270   5.961538
    2      3.563717  7.698389e+01    4.272541     0.507196   5.961538
    3      8.077432  7.751301e+01   12.426412     7.590147   5.961538
    4     12.944304  7.328047e+01   27.727650    49.393532   5.961538
    5     15.455938  6.539441e+01   48.514023   157.940778   5.961538
    6     15.559567  5.632780e+01   71.074793   323.988034   5.961538
    7     14.213901  4.792987e+01   92.634726   515.047542   5.961538
    8     12.175182  4.120771e+01  111.681695   702.794957   5.961538
    9      9.967967  3.655616e+01  127.621874   870.375762   5.961538
    10     7.904553  3.397030e+01  140.461226  1010.995856   0.000000
    11     6.130542  2.971557e+01  150.527337  1124.231868   0.000000
    12     4.684648  2.455938e+01  158.275914  1212.754191   0.000000
    13     3.539414  2.064294e+01  164.163590  1280.601954   0.000000
    14     2.650958  1.769791e+01  168.592688  1331.862018   0.000000
3   0      0.596154  3.961538e+01    0.000000     0.000000   0.000000
    1      1.859281  3.854370e+01    1.609582     0.071448   0.000000
    2      5.082063  3.542097e+01    6.300022     1.621073  19.423077
    3     10.697473  3.920078e+01   17.454464    18.322335  19.423077
    4     16.174558  4.557242e+01   37.054410    93.617634  19.423077
    5     18.483486  4.808087e+01   62.453319   257.859917  19.423077
    6     17.584114  5.000668e+01   88.726028   480.565097  19.423077
    7     14.900189  5.376454e+01  112.246631   710.911454  19.423077
    8     11.566333  6.047187e+01  131.337077   913.806911  19.423077
    9      8.126217  7.045323e+01  145.510584  1071.950701   0.000000
    10     5.400009  6.5

## Analyse single experiment

Here you can visualize particular run from the generated dataset. You can change the `select_run_ix` to decide which run will be plotted.



In [17]:
""" Select specific run to be plotted """
PLOT_RUN_ID = 27
""" Select group of run to be plotted"""
PLOT_RUNS_IDS = [0,1,2,3,4,5,6,7,8,9] 

In [18]:
# Evolution profiles for selected runs
if PLOT_RUN_ID is not None:
    fig = make_subplots(rows=1, cols=5, subplot_titles=owu.columns)
    plot_run_ix = owu.index.get_level_values("run") == PLOT_RUN_ID
    for i,c in enumerate(owu.columns):
        fig.add_trace(go.Scatter(x=list(range(15)), y=owu[c].values[plot_run_ix]), row=1, col=i+1)
    fig.update_layout(showlegend=False, title_text="X Variable evolution for run "+str(PLOT_RUN_ID))
    fig.show()
if PLOT_RUNS_IDS is not None:
    fig = make_subplots(rows=1, cols=5, subplot_titles=owu.columns)
    for j in PLOT_RUNS_IDS:
        plot_run_ix = owu.index.get_level_values("run") == j
        for i,c in enumerate(owu.columns):
            fig.add_trace(go.Scatter(x=list(range(15)), y=owu[c].values[plot_run_ix], name="Run id = " + str(j),marker=dict(color=px.colors.qualitative.Plotly[j % 10])), row=1, col=i+1)
    fig.update_layout(showlegend=False, title_text="X Variable evolution for runs "+str(PLOT_RUNS_IDS))
    fig.show()


## Analyse all experiments

Here you can visualize all run from the generated dataset. By changing the `select_color` you decide on the coloring of the experiments. The options are:
* `Run_id` runs are colored by the order in which they appear in the dataset.
* `Titer_14` runs are colored by the amount of Titer at day 14 or the experiments.
* `Glc_0` run are colored by the designed initial Glucose level
* `VCD_0` run are colored by the designed initial VCD level
* `feed_start` run are colored by the designed feeding start day 
* `feed_end` run are colored by the designed feeding end day
* `Glc_feed_rate` run are colored by the designed Glucose feed rate

In [21]:
""" Plot coloring option, one of (Run_id, Titer_14, Glc_0, VCD_0, feed_start, feed_end, Glc_feed_rate)"""
PLOT_COLOR = "Titer_14"
""" Select run to be highlighted in all plots """
PLOT_HIGHLIGHT = 10

In [23]:
# Plot all Univariate plots
color_show=True
if PLOT_COLOR == "Titer_14":
    color_idx =np.repeat(np.array(owu["X:Titer"][:,14]),15)
elif PLOT_COLOR in doe.columns:
    color_idx = np.repeat(np.array(doe[PLOT_COLOR]),15)
else:
    PLOT_COLOR ="Run_id"
    color_idx=np.repeat(np.array(list(range(doe.shape[0]))),15)     
    color_show=False
owu["color"] = color_idx

for i in owu.columns[0:-1]:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=owu.index.get_level_values(1),y=owu[i],mode='markers',marker=dict(size=0,color="rgba(0,0,0,0)",colorscale='Portland',cmin=min(color_idx),cmax=max(color_idx),colorbar=dict(thickness=40, title=str(PLOT_COLOR))),showlegend=False))
    for color_val in np.unique(color_idx):
        color_val_norm = (color_val -min(color_idx)) / (max(color_idx)-min(color_idx))
        owu_subset = owu[owu['color']==color_val]
        fig.add_trace(go.Scatter(x=owu_subset.index.get_level_values(1),y=owu_subset[i],mode='lines+markers',name="Run id = " + str(owu.index.get_level_values(0)[color_val == color_idx][0]),marker=dict(color=plothelpers.get_color('Portland',color_val_norm))))
    if PLOT_HIGHLIGHT is not None:
        highlight_ix = owu.index.get_level_values(0)==PLOT_HIGHLIGHT
        fig.add_trace(go.Scatter(x=owu_subset.index.get_level_values(1),y=owu[i][highlight_ix],mode='lines+markers',name="Run id = " + str(PLOT_HIGHLIGHT),marker=dict(color="black",size=10)))              
    fig.update_layout(showlegend=False,xaxis_title='days',yaxis_title=i, title=i)
    fig.show()
if 'color' in owu.columns: owu.drop(columns='color',inplace=True)

# Tasks

1. Identify which process parameters produce the highest level of titer in the simulated experiments.
2. Replicate it in single experiment generation and try to modify it to produce even higher level of Titer.